In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

## IMPORT FROM DATASET
import load_image_batch as driving_data

In [2]:
def conv2d(x, W):
    return tf.nn.conv2d(
        x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(
        x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [3]:
# Input and output variables

INPUTS = 60 * 200 * 3
OUTPUTS = 1
BATCH_SIZE = 30
NUM_EPOCHS = 50
LEARNING_RATE = 1e-04
NUM_IMAGES = 42000

In [4]:
try:
    sess.close()
except:
    pass

sess = tf.InteractiveSession()

In [ ]:
# Input and output placeholder
x_image = tf.placeholder(tf.float32, shape=[None, 64, 224, 3])
y = tf.placeholder(tf.float32, shape=[None, OUTPUTS])
pkeep = tf.placeholder(tf.float32)

print x_image.get_shape()
# First Convolutional Layer
W_conv_1 = tf.Variable(tf.truncated_normal([2, 2, 3, 64], stddev=0.1))
b_conv_1 = tf.Variable(tf.constant(0.0, shape=[64]))
h_conv_1 = tf.nn.relu(conv2d(x_image, W_conv_1) + b_conv_1)
h_pool_1 = max_pool_2x2(h_conv_1)

print(h_pool_1.get_shape())

# Second Convolutional Layer
W_conv_2 = tf.Variable(tf.truncated_normal([2, 2, 64, 128], stddev=0.1))
b_conv_2 = tf.Variable(tf.constant(0.0, shape=[128]))
h_conv_2 = tf.nn.relu(conv2d(h_pool_1, W_conv_2) + b_conv_2)
h_pool_2 = max_pool_2x2(h_conv_2)

print(h_pool_2.get_shape())

# Third Convolutional Layer
W_conv_3 = tf.Variable(tf.truncated_normal([2, 2, 128, 256], stddev=0.1))
b_conv_3 = tf.Variable(tf.constant(0.0, shape=[256]))
h_conv_3 = tf.nn.relu(conv2d(h_pool_2, W_conv_3) + b_conv_3)
h_pool_3 = max_pool_2x2(h_conv_3)

print(h_pool_3.get_shape())

# Fourth Convolutional Layer
W_conv_4 = tf.Variable(tf.truncated_normal([2, 2, 256, 512], stddev=0.1))
b_conv_4 = tf.Variable(tf.constant(0.0, shape=[512]))
h_conv_4 = tf.nn.relu(conv2d(h_pool_3, W_conv_4) + b_conv_4)
h_pool_4 = max_pool_2x2(h_conv_4)

print(h_pool_4.get_shape())

# Densely connected layer
W_fc1 = tf.Variable(tf.truncated_normal([4 * 14 * 512, 128], stddev=0.1))
b_fc1 = tf.Variable(tf.constant(0.0, shape=[128]))
h_poolfc1_flat = tf.reshape(h_pool_4, [-1, 4 * 14 * 512])
h_fc1 = tf.nn.relu(tf.matmul(h_poolfc1_flat, W_fc1) + b_fc1)

# Densely connected layer
W_fc2 = tf.Variable(tf.truncated_normal([128, 256], stddev=0.1))
b_fc2 = tf.Variable(tf.constant(0.0, shape=[256]))
h_fc2 = tf.nn.relu(tf.matmul(h_fc1, W_fc2) + b_fc2)

# Dropout
h_drop = tf.nn.dropout(h_fc2, pkeep)

# Read out Layer
W_fc3 = tf.Variable(tf.truncated_normal([256, OUTPUTS], stddev=0.1))
b_fc3 = tf.Variable(tf.constant(0.0, shape=[OUTPUTS]))
y_logits = tf.matmul(h_drop, W_fc3) + b_fc3
y_softmax = tf.nn.softmax(y_logits)

loss = tf.sqrt(tf.reduce_mean(tf.square(y - y_softmax)), name='RMSE')
optimizer = tf.train.AdamOptimizer(LEARNING_RATE)
train_step = optimizer.minimize(loss)

correct_prediction = tf.sub(tf.cast(1, 'float'), tf.reduce_mean(tf.sub(y_softmax, y)))
accuracy = tf.cast(correct_prediction, "float")

init = tf.initialize_all_variables()

saver = tf.train.Saver()

sess.run(init)

(?, 64, 224, 3)
(?, 32, 112, 64)
(?, 16, 56, 128)
(?, 8, 28, 256)
(?, 4, 14, 512)


In [ ]:
loss_train_array = []
test_accuracy_array = []
train_accuracy_array = []

for current_epoch in range(NUM_EPOCHS):
    
    test_dataset, test_labels = driving_data.get_test_batch(BATCH_SIZE)

    for step in xrange(1):
        
        train_dataset, train_labels = driving_data.get_train_batch(BATCH_SIZE)
        
        # This dictionary maps the batch data (as a numpy array) to the
        # node in the graph is should be fed to.
        feed_dict = {x_image: train_dataset, y: train_labels, pkeep: 0.8}
        _, loss_train = sess.run([train_step, loss],
                                 feed_dict=feed_dict)

    # We calculate the accuracies to plot their values later
    loss_train_array.append(loss_train)
    
    train_accuracy = sess.run(
        accuracy, feed_dict={x_image: train_dataset, y: train_labels, pkeep: 1.0})
    
    train_accuracy_array.append(train_accuracy)
    
    test_accuracy = sess.run(
        accuracy, feed_dict={x_image: test_dataset, y: test_labels, pkeep: 1.0})
    
    test_accuracy_array.append(test_accuracy)

    print (
        'Epoch %04d, '
        'loss train %.8f, train accuracy %.8f, test accuracy %.8f'
        %
        (current_epoch,
         loss_train,
         train_accuracy,
         test_accuracy))

Epoch 0000, loss train 20.26641655, train accuracy 1.97699976, test accuracy -3.75766659
Epoch 0001, loss train 33.69781494, train accuracy -1.82466578, test accuracy 3.63999987
Epoch 0002, loss train 54.55800629, train accuracy -6.32033348, test accuracy -3.02199984
Epoch 0003, loss train 18.46153831, train accuracy 0.12100017, test accuracy -4.05466604
Epoch 0004, loss train 18.89205551, train accuracy -0.73933339, test accuracy -1.55233312
Epoch 0005, loss train 21.17448807, train accuracy -2.96866655, test accuracy 1.19966650
Epoch 0006, loss train 19.08725166, train accuracy -3.38100004, test accuracy -6.69266653
